In [1]:
from dotenv import load_dotenv
import os
from langchain.llms import OpenAI
load_dotenv()
import warnings
warnings.filterwarnings("ignore")

In [2]:
## temperature value --> how creative we want our model to be
## 0 --> temperature it means model is very safe it is not taking any bets.
## 1 --> temperature  it will take risk it might generate wrong output but it is very creative.

In [3]:
llm = OpenAI(openai_api_key=os.getenv("openai_key"),temperature=0.7)

In [4]:
text = '''Which are the five best places in India and i want point wise answer i do not want
detail description i just want names'''
print(llm.predict(text))




1. Taj Mahal, Agra
2. Golden Temple, Amritsar
3. Jaipur, Rajasthan
4. Goa
5. Kerala Backwaters


## Prompt Templates and LLMChain

In [5]:
from langchain.prompts import PromptTemplate

input_prompt = PromptTemplate(
    input_variables=['country'],
    template = '''Give me the names of 5 best movies of {country} with highest box office collection 
    and also give the total worldwide collection in INR'''
)

print(input_prompt.format(country='India'))

Give me the names of 5 best movies of India with highest box office collection 
    and also give the total worldwide collection in INR


In [6]:
# In order to execute a prompt template you need to use LLMChain
from langchain.chains import LLMChain
chain = LLMChain(llm=llm,prompt=input_prompt)
print(chain.run("Korea"))



1. "Parasite" - Worldwide collection: INR 441 crore
2. "Train to Busan" - Worldwide collection: INR 312 crore
3. "Along with the Gods: The Two Worlds" - Worldwide collection: INR 271 crore
4. "The Admiral: Roaring Currents" - Worldwide collection: INR 251 crore
5. "Veteran" - Worldwide collection: INR 224 crore


## Combining Multiple Chains using Sequential Chain

In [7]:
# Simple Sequential Chain
# from langchain.chains import SimpleSequentialChain

# country_prompt = PromptTemplate(input_variables=['country'],
#                                 template='''Give me the name of richest man of {country} ''')
# country_chain = LLMChain(llm=llm,prompt=country_prompt,output_key='Person_Name')

# networth_prompt = PromptTemplate(input_variables=['networth'],
#                                  template='''And also tell me the networth of that person is {networth} ''')
# networth_chain = LLMChain(llm=llm,prompt=networth_prompt,output_key='Net_Worth')

In [8]:
# main_chain = SimpleSequentialChain(chains=[country_chain,networth_chain])
# main_chain.run("US")

In [9]:
# Sequential Chain
from langchain.chains import SequentialChain
country_prompt = PromptTemplate(input_variables=['country'],
                                template='''Tell me the richest man of {country} and i just want one name only ''')
country_chain = LLMChain(llm=llm,prompt=country_prompt,output_key='name')

networth_prompt = PromptTemplate(input_variables=['name'],
                                 template='''net worth of the {name}''')
networth_chain = LLMChain(llm=llm,prompt=networth_prompt,output_key='Net_Worth')


my_chain = SequentialChain(chains=[country_chain,networth_chain],
                        input_variables=['country'],
                        output_variables=['name','Net_Worth'])


In [10]:
my_chain({'country':'USA'})

{'country': 'USA',
 'name': '\n\nAs of 2021, the richest man in the USA is Elon Musk, with a net worth of approximately $177 billion. ',
 'Net_Worth': ''}

## Chatmodels with ChatOpenAI

In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

In [12]:
chat_llm = ChatOpenAI(openai_api_key=os.getenv("openai_key"),temperature=0.7,model='gpt-3.5-turbo')

In [13]:
chat_llm([
    SystemMessage(content="You are a professional singer AI assistant"),
    HumanMessage(content="Please write a song on Lord Hanuman")
])

AIMessage(content='(Verse 1)\nIn the realms of ancient lore, a tale unfolds\nOf a deity with strength untold\nLord Hanuman, mighty and wise\nWith devotion that reaches the skies\n\n(Chorus)\nOh Hanuman, the son of the wind\nProtector of the weak, a true friend\nWith your courage and love so grand\nYou hold the power to save our land\n\n(Verse 2)\nBorn to Anjana, a celestial being\nBlessed by Vayu, the wind king\nYou possess the speed of the breeze\nAnd the strength to bring enemies to their knees\n\n(Chorus)\nOh Hanuman, the son of the wind\nProtector of the weak, a true friend\nWith your courage and love so grand\nYou hold the power to save our land\n\n(Bridge)\nIn the search for Sita, you flew across the sea\nTo Lanka, where Ravana held her captive, you see\nYou leapt across the abyss, fearless and strong\nWith devotion, you righted every wrong\n\n(Verse 3)\nWith a heart full of devotion, you bring solace\nTo those who seek your divine grace\nYour loyalty to Lord Rama, unwavering and

## Prompt Template + LLM + Output Parsers

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [15]:
class Commaseparatedoutput(BaseOutputParser):
    def parse(self,text:str):
        return text.strip().split('\n')

In [16]:
template = '''
You are a Data Science helpful assistant. So when the user asks you something about data science,machine learning,
deep learning, NLP and generative AI so on this topics you have to guide them properly   
'''

human_template = '{text}'

chat_prompt = ChatPromptTemplate.from_messages([
    ('system',template),
    ('human',human_template)
])

In [17]:
final_chain = chat_prompt|chat_llm|Commaseparatedoutput()

In [18]:
final_chain.invoke({'text':'Roadmap for machine learning'})

['Sure! Here is a general roadmap for getting started with machine learning:',
 '',
 '1. Learn Python: Python is the most widely used programming language for machine learning. Start by learning the basics of Python and familiarize yourself with libraries like NumPy and Pandas, which are commonly used for data manipulation and analysis.',
 '',
 '2. Understand the Fundamentals of Machine Learning: Start by understanding the basic concepts of machine learning, such as supervised learning, unsupervised learning, and reinforcement learning. Familiarize yourself with common terms like training data, testing data, features, labels, and models.',
 '',
 '3. Learn Data Preprocessing: Data preprocessing is an important step in machine learning. Learn about techniques like data cleaning, feature scaling, handling missing values, and encoding categorical variables.',
 '',
 '4. Explore Machine Learning Algorithms: There are various types of machine learning algorithms, including linear regression, 